In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import pandas as pd
import numpy as np
import py_entitymatching as em # install from https://pypi.org/project/py-entitymatching/
import datetime

In [ ]:
print(datetime.datetime.now())

In [2]:
def remove(string):
    if pd.isnull(string):
        return np.nan
    else:
        return string.replace(" ","").lstrip().rstrip()


In [3]:
def calc_set_intersection(text_a,text_b):
    try:
        text_a = str(text_a)
        a = set(text_a.split())
        b = set(text_b.split())
    except:
        print(text_a)
        print(text_b)
        
    return len(a.intersection(b))*(1.0)

In [ ]:
result_MultiBR = pd.read_csv(INPUT_FILE_DUNS_MULTIPLE_ENTITY_ID) # They were approx 45000 early last year
print(result_MultiBR.shape)

In [ ]:
data1 = pd.read_csv(CSV_FILE_FROM_AMY_SNOWFLAKE_CODE_PART1)
data2 = pd.read_csv(CSV_FILE_FROM_AMY_SNOWFLAKE_CODE_PART2) # files are split as part1 and part2 because of size

data = pd.concat([data1,data2],axis =0)

print(data1.shape)
print(data2.shape)
print(data.shape)


In [ ]:
sub = pd.read_csv(FILENAME_TRADE_UCC_SEGMENT)
print(sub.shape)

In [ ]:
sub = pd.merge(sub, result_MultiBR[['ENTITY_ID','BUSINESS_ENTITY_ID','IDENTIFICATION_NUMBER',\
                                   'PR_DUNS','FILING_OFFICE_STATE_ABBREVIATED_NAME']],on = 'ENTITY_ID',\
                                   how = 'inner',suffixes = ['_sub','_r'])

print(sub.shape)


In [ ]:
sub['in'] = sub[['BUSINESS_ENTITY_ID_sub','BUSINESS_ENTITY_ID_r']].apply(lambda row: row['BUSINESS_ENTITY_ID_sub'].astype(str) in row['BUSINESS_ENTITy_ID_r'].astype(str))

In [ ]:
sub.sort_values(by = 'in')

In [ ]:
data.rename(columns = {'PRIMARY_BUSINESS_NAME_GAA_B':'PRIMARY_BUSINESS_NAME_GAA_B_old',
                       'PHYSICAL_ADDRESS_STATE__GAA_X':'PHYSICAL_ADDRESS_STATE__GAA_X_old'},inplace=True)


In [ ]:
print(len(set(sub['BUSINESS_ENTITY_ID_r'])))
print(len(set(data['PR_DUNS'])))
print(data.shape)
data = data[data['PR_DUNS'].isin(sub['BUSINESS_ENTITY_ID_r'])]
print(data.shape)
print(len(set(data['PR_DUNS'])))

In [ ]:
data['verify'] = np.nan

In [ ]:
data.rename(columns = {'PRIMARY_BUSINESS_NAME_GAA_B':'PRIMARY_BUSINESS_NAME_GAA_B'},inplace=True)

data['PRIMARY_BUSINESS_NAME_GAA_B'] = data['PRIMARY_BUSINESS_NAME_GAA_B'].str.upper()
data['PRIMARY_BUSINESS_NAME_GAA_B'] = data['PRIMARY_BUSINESS_NAME_GAA_B'].str.upper()


In [ ]:
data['PRIMARY_BUSINESS_NAME_GAA_B'] = data['PRIMARY_BUSINESS_NAME_GAA_B'].str.upper()
data['BUS_NAME'] = data['BUS_NAME'].str.upper()
data['PRIMARY_BUSINESS_NAME_GAA_B'] = data['PRIMARY_BUSINESS_NAME_GAA_B'].str.upper()
data['PRIMARY_BUSINESS_NAME_GAA_B_orig'] = data['PRIMARY_BUSINESS_NAME_GAA_B'].copy()


In [ ]:
data['PRIMARY_BUSINESS_NAME_GAA_B'] = data['PRIMARY_BUSINESS_NAME_GAA_B'].str.replace(',','')
data['PRIMARY_BUSINESS_NAME_GAA_B'] = data['PRIMARY_BUSINESS_NAME_GAA_B'].str.replace('.','')
data['PRIMARY_BUSINESS_NAME_GAA_B'] = data.apply(lambda row: remove(row['PRIMARY_BUSINESS_NAME_GAA_B']),axis =1)

data['BUS_NAME_temp'] = data['BUS_NAME'].str.replace(',','')
data['BUS_NAME_temp'] = data['BUS_NAME'].str.replace('.','')
data['BUS_NAME_temp'] = data.apply(lambda row: remove(row['BUS_NAME']),axis =1)


In [ ]:
data.loc[(data['PRIMARY_BUSINESS_NAME_GAA_B'].notnull())\
         &(data['PRIMARY_BUSINESS_NAME_GAA_B'].str.contains('THE$')),'PRIMARY_BUSINESS_NAME_GAA_B'] = 'THE'+ data.loc[(data['PRIMARY_BUSINESS_NAME_GAA_B'].notnull())\
         &(data['PRIMARY_BUSINESS_NAME_GAA_B'].str.contains('THE$')),'PRIMARY_BUSINESS_NAME_GAA_B'].str[:-3]

In [ ]:
data.loc[data['SOURCE']=='MULTI_BR','jw1a'] = data.loc[data['SOURCE']=='MULTI_BR'].apply(lambda row: em.jaro_winkler('PRIMARY_BUSINESS_NAME_GAA_B_orig','BUS_NAME'),axis = 1)
data.loc[data['SOURCE']=='MULTI_BR','jw1b'] = data.loc[data['SOURCE']=='MULTI_BR'].apply(lambda row: em.jaro_winkler('PRIMARY_BUSINESS_NAME_GAA_B','BUS_NAME_temp'),axis = 1)

In [ ]:
data_jw1b = pd.DataFrame(data.loc[(data['jw1b']==1.0)&(data['jw1a']!= 1.0)]).groupby(['PR_DUNS'])['jw1b'].count()
data_jw1b.reset_index(drop =False,inplace=True)
print(data_jw1b.shape)
after_clean_duns = data_jw1b.loc[(data_jw1b['jw1b']==1),'PR_DUNS'].values
print(data_jw1b.shape)


In [ ]:
data.loc[(data['jw1b']!= 1.0)&(~(data['PR_DUNS'].isin(after_clean_duns))),'jw1'] = data.loc[(data['jw1b']!= 1.0)&(~(data['PR_DUNS'].isin(after_clean_duns))),['jw1a','jw1b']].max(axis = 1)
data.loc[(data['jw1b']!= 1.0)&((data['PR_DUNS'].isin(after_clean_duns))),'jw1']  = data.loc[(data['jw1b']!= 1.0)&((data['PR_DUNS'].isin(after_clean_duns))),'jw1a']
data.loc[(data['PR_DUNS'].isin(after_clean_duns)),'jw1'] = data.loc[(data['PR_DUNS'].isin(after_clean_duns)),['jw1a','jw1b']].max(axis = 1)

In [ ]:
data.drop(columns = ['BUS_NAME_temp'],inplace=True)

In [ ]:
data[['rank_jw1','rank_jw2']]= data.groupby(['PR_DUNS'])['jw1','jw1a'].rank(ascending = False)
data['rank1'] = data.groupby(['PR_DUNS'])['jw1'].rank(ascending = False)

In [ ]:
data['rank1'].value_counts()

In [ ]:
data = data.sort_values(by = ['PR_DUNS','DATA_SOURCE','rank1'],ascending = [1,1,0])

In [ ]:
data = data.sort_values(by = ['PR_DUNS','DATA_SOURCE','rank_jw1','rank_jw2'],ascending = [1,1,0,0])

In [ ]:
data.loc[:,'commonwords'] = data[(data['PRIMARY_BUSINESS_NAME_GAA_B_orig'].notnull())&(data['BUS_NAME'].notnull())].apply(lambda row: calc_set_intersection(data['PRIMARY_BUSINESS_NAME_GAA_B_orig'],data['BUS_NAME']),axis = 1)

In [ ]:
print(len(set(data['PR_DUNS'])))
print(data[data['DATA_SOURCE']=='MULTI_BR'].shape)

In [ ]:
ucc = pd.read_csv(FILE_CONTAINS_DUNS_WITH_UCC_ATTACHED)
print(ucc.shape)

In [ ]:
suits = pd.read_csv(FILE_CONTAINS_DUNS_WITH_SUITS_ATTACHED)
print(suits.shape)

In [ ]:
duns_liens_judgements = pd.read_csv(FILE_CONTAINS_DUNS_WITH_DUNS_LIENS_JUDGEMENTS_ATTACHED)
print(duns_liens_judgements.shape)

In [ ]:
bankruptcy = pd.read_csv(FILE_CONTAINS_DUNS_WITH_BANKRUPTCY_ATTACHED)
print(bankruptcy.shape)

In [ ]:
LIEN_PR_DUNS = data.loc[data['DATA_SOURCE'] == 'LIEN','PR_DUNS']
JUDG_PR_DUNS = data.loc[data['DATA_SOURCE'] == 'JUDG','PR_DUNS']

In [ ]:
## Remove duns that have bankruptcy, suits , liens , UCC
print(data.shape)
data = data[~data['PR_DUNS'].isin(duns_liens_judgements['PR_DUNS'])]
data = data[~data['PR_DUNS'].isin(LIEN_PR_DUNS)]
data = data[~data['PR_DUNS'].isin(JUDG_PR_DUNS)]
print(data.shape)

In [ ]:
print(len(set(data['PR_DUNS'])))
print(data[data['DATA_SOURCE'] == 'MULTI_BR'].shape)

In [ ]:
set(data['DATA_SOURCE'])

In [ ]:
SUIT_PR_DUNS = data.loc[data['DATA_SOURCE']== 'SUIT','PR_DUNS']


In [ ]:
print(data.shape)
data = data[~(data['PR_DUNS'].isin(suits['BUS_DUNS_NBR']))]
data = data[~(data['PR_DUNS'].isin(SUIT_PR_DUNS))]
print(data.shape)

In [ ]:
bankruptcy_PR_DUNS = data.loc[data['DATA_SOURCE']== 'BANKRUPTCY','PR_DUNS']


In [ ]:
print(data.shape)
data = data[~(data['PR_DUNS'].isin(bankruptcy['BUS_DUNS_NBR']))]
data = data[~(data['PR_DUNS'].isin(bankruptcy_PR_DUNS))]
print(data.shape)

In [ ]:
UCC_PR_DUNS = data.loc[data['DATA_SOURCE']== 'UCC','PR_DUNS']


In [ ]:
print(data.shape)
data = data[~(data['PR_DUNS'].isin(ucc['BUS_DUNS_NBR']))]
data = data[~(data['PR_DUNS'].isin(UCC_PR_DUNS))]
print(data.shape)

In [ ]:
data['shift_data']    = data['DATA_SOURCE'].shift(1)
data['lead_bus_name'] = data.groupby(['PR_DUNS'])['BUS_NAME'].shift(1)
data['lag_bus_name']  = data.groupby(['PR_DUNS'])['BUS_NAME'].shift(-1)

In [ ]:
data[(data['shift_data']==data['DATA_SOURCE'])&(data['DATA_SOURCE'] == 'MULTi_BR')&\
    (data['BUS_NAME'] == data['lead_bus_name'])].shape

In [ ]:
br_duns_samename_only = data[(data['shift_data']==data['DATA_SOURCE'])&(data['DATA_SOURCE'] == 'MULTi_BR')&\
    (data['BUS_NAME'] == data['lead_bus_name'])]

print(br_duns_samename_only.shape)
print(br_duns_samename_only[br_duns_samename_only['DATA_SOURCE']=='MULTI_BR'].shape)
print(len(set(br_duns_samename_only['PR_DUNS'])))

In [ ]:
print(len(br_duns_samename_only['PR_DUNS'])[0:15])

In [ ]:
print(data.shape)
data_same = data[data['PR_DUNS'].isin(br_duns_samename_only['PR_DUNS'])]
print(data_same.shape)


In [ ]:
data_same.columns

In [ ]:
sum(data_same['DATA_SOURCE'].isnull())

In [ ]:
data_same_duns = data_same[data_same['DATA_SOURCE']=='MULTI_BR'].groupby(['PR_DUNS'])
data_same_duns = pd.DataFrame(data_same_duns)
data_same_duns.reset_index(drop = False,inplace=True)
data_same_duns.columns = ['PR_DUNS','count']

In [ ]:
data_same_duns_2 = data_same_duns[data_same_duns['count']==2]
data_same_duns_plus = data_same_duns[data_same_duns['count']>2]
data_same_duns_plus

In [ ]:
print(data.shape)
data = data[~data['PR_DUNS'].isin(br_duns_samename_only['PR_DUNS'])]
print(data.shape)

In [ ]:
print(len(set(data['PR_DUNS'])))
print(data[data['DATA_SOURCE']=='MULTI_BR'].shape)

In [ ]:
data['rank1'].value_counts().head(50)

In [ ]:
data['ACTION_FINAL'].value_counts()

In [ ]:
data[['ACCT_ID','TRD_REF_NBR','SUPP_ID']] = np.nan

In [ ]:
us_trade_A = pd.read_csv(TRADE_FILE_PART1) # TRADE file is split because of file size
us_trade_B = pd.read_csv(TRADE_FILE_PART2)

us_trade = pd.concat([us_trade_A,us_trade_B],axis = 0)
print(us_trade_A.shape)
print(us_trade_B.shape)
print(us_trade.shape)

print(us_trade.columns)

cols_to_keep_trade = ['BUSINESS_DUNS_NUMBER','PRIM_NME','LN_1_ADDR','PRIM_TOWN_NME','TRD_REF_NBR','SUPP_ID'
                     ]

us_trade_short = us_trade[cols_to_keep_trade]

us_trade_short.columns = ['PR_DUNS','BUS_NAME','BUS_STREET','BUS_CITY','BUS_STATE','ACCT_ID','TRD_REF_NBR','SUPP_ID']
us_trade_short['DATA_SOURCE'] = 'TRADE'
us_trade_short['DUNS_NUMBER_WAA_D'] = us_trade_short['PR_DUNS']
print(us_trade_short.shape)



In [ ]:
us_trade_short[['PRIMARY_BUSINESS_NAME_GAA_B','PHYSICAL_ADDRESS_STREET1_GAA_S','PHYSICAL_ADDRESS_STREET1_GAA_U',
                'PHYSICAL_ADDRESS_CITY_GAA_W','PHYSICAL_ADDRESS_STATE__GAA_X','CEO_GAA_J','ACTION_FINAL','BUSREG_STATUS_DESP','FILG_DT','IDENTIFICATION_NUMBER','FILING_OFFICE_STATE_ABBREVIATED_NAME','AMEND_FILE_DT','AMEND_DESC','FINAL_SCORE','PR_SCORE_NORM','BR_SCORE_NORM','TRADE_SCORE_NORM','DUNS_SCORE_NORM','shift_data','lead_bus_name','lag_bus_name']] = np.nan

In [ ]:
print(us_trade_short.shape)
print(data.columns)

In [ ]:
print(us_trade_short.shape)
us_trade_short = us_trade_short[us_trade_short['PR_DUNS'].isin(data['PR_DUNS'])]
print(us_trade_short.shape)

In [ ]:
print(data.shape)
data = pd.concat([us_trade_short,data],axis = 0)
print(data.shape)


In [ ]:
data.columns

In [ ]:
batchmatch1 = pd.read_csv(BATCHMATCH_FILE1) ## split batchmatch files because are large
batchmatch2 = pd.read_csv(BATCHMATCH_FILE2)
batchmatch3 = pd.read_csv(BATCHMATCH_FILE3)

batchmatch = pd.concat([batchmatch1,batchmatch2,batchmatch3],axis = 0)
print(batchmatch1.shape)
print(batchmatch2.shape)
print(batchmatch3.shape)
print(batchmatch.columns)

In [ ]:
cols_to_keep = ['BUSINESS_DUNS_NUMBER','IN_NAME']

In [ ]:
batchmatch_short = batchmatch['cols_to_keep']

In [ ]:
batchmatch_short.columns = ['PR_DUNS','BUS_NAME']

In [ ]:
batchmatch_short['DATA_SOURCE']  = 'ZBATCHMATCH'

In [ ]:
batchmatch_short['DUNS_NUMBER_WAA_D'] = batchmatch_short['PR_DUNS']

In [ ]:
batchmatch_short[['PHYSICAL_ADDRESS_STREET1_GAA_S','PHYSICAL_ADDRESS_STREET1_GAA_U',
                'PHYSICAL_ADDRESS_CITY_GAA_W','PHYSICAL_ADDRESS_STATE__GAA_X','CEO_GAA_J','ACTION_FINAL','BUSREG_STATUS_DESP','FILG_DT','IDENTIFICATION_NUMBER','FILING_OFFICE_STATE_ABBREVIATED_NAME','AMEND_FILE_DT','AMEND_DESC','FINAL_SCORE','PR_SCORE_NORM','BR_SCORE_NORM','TRADE_SCORE_NORM','DUNS_SCORE_NORM','shift_data','lead_bus_name','lag_bus_name','ACCT_ID','TRD_REF_NBR','SUPP_ID']] = np.nan

In [ ]:
print(batchmatch_short.shape)
batchmatch_short = batchmatch_short[batchmatch_short['PR_DUNS'].isin(data['PR_DUNS'])]
print(batchmatch_short.shape)


In [ ]:
print(data.shape)
data = pd.concat([batchmatch_short,data],axis = 0)
print(data.shape)

In [ ]:
print(len(set(data.loc[data['PR_DUNS'].notnull(),'PR_DUNS'])))

In [ ]:
print(data.shape)
data = data.sort_values(by = ['PR_DUNS','rank1'])
print(data.shape)

In [ ]:
data_good_gby = data.groupby(['PR_DUNS','DATA_SOURCE'])['jw1'].agg(['max'])
data_good_gby.reset_index(drop = False, inplace=True)
data_good_gby

In [ ]:
print(data.shape)
data = pd.merge(data,data_good_gby,how = 'left')
print(data.shape)


In [ ]:
data['match'] = data['jw1'] == data['max']

In [ ]:
data_good_bronly = data.loc[data['DATA_SOURCE']=='MULTI_BR'][['PR_DUNS','BUS_NAME','rank1']]


In [ ]:
data_good_bronly = good_good_bronly.sort_values(by = ['PR_DUNS','rank1'])
data_good_bronly.drop(columns = 'rank1',inplace=True)

In [ ]:
data_good_bronly['Br_count'] = data_good_bronly.groupby(['PR_DUNS']).cumcount()+1


In [ ]:
data_good_bronly = data_good_bronly[data_good_bronly['PR_DUNS']! = 191919799.0]


In [ ]:
data_good_bronly_pivot = data_good_bronly[data_good_bronly['PR_DUNS'].notnull()].pivot(index = 'PR_DUNS', coulmns = 'Br_count')

In [ ]:
data_good_bronly_pivot.columns = ['BUS_NAME_'+str(cols[1])] for cols in data_good_bronly_pivot.colmns]

In [ ]:
data_good_bronly_pivot.reset_index(drop = False, inplace = True)


In [ ]:
data_good_bronly_pivot.columns


In [ ]:
data['rank1'].value_counts().head(50)


In [ ]:
import gc
gc.collect()

In [ ]:
data['rank1'].value_counts().head(50)


In [ ]:
data['BUS_NAME_clean'] = data['BUS_NAME'].str.replace(',','')
data['BUS_NAME_clean'] = data['BUS_NAME_clean'].str.replace('.','')

data['BUS_NAME_1clean'] = data['BUS_NAME_1'].str.replace(',','')
data['BUS_NAME_1clean'] = data['BUS_NAME_1clean'].str.replace('.','')


data['BUS_NAME_2clean'] = data['BUS_NAME_2'].str.replace(',','')
data['BUS_NAME_2clean'] = data['BUS_NAME_2clean'].str.replace('.','')

In [ ]:
data.loc[:,'jw_trade1a'] = data.apply(lambda row: em.jaro_wrinkler(row['BUS_NAME_1clean'],row['BUS_NAME_clean']),axis = 1)
data.loc[:,'jw_trade2a'] = data.apply(lambda row: em.jaro_wrinkler(row['BUS_NAME_2clean'],row['BUS_NAME_clean']),axis = 1)

In [ ]:
def remove(string):
    if pd.isnull(string):
        return np.nan
    else:
        return string.replace(" ","").lstrip().rstrip()

In [ ]:
data['BUS_NAME_clean_b'] = data.apply(lambda row: remove(row['BUS_NAME_clean']),axis = 1 )
data['BUS_NAME_1clean_b'] = data.apply(lambda row: remove(row['BUS_NAME_1clean']),axis = 1 )
data['BUS_NAME_2clean_b'] = data.apply(lambda row: remove(row['BUS_NAME_2clean']),axis = 1 )

In [ ]:
data.loc[:,'jw_trade1b'] = data.apply(lambda row:em.jaro_winkler(row['BUS_NAME_1clean_b'], row['BUS_NAME_clean_b']),axis = 1)
data.loc[:,'jw_trade2b'] = data.apply(lambda row:em.jaro_winkler(row['BUS_NAME_2clean_b'], row['BUS_NAME_clean_b']),axis = 1)


In [ ]:
data.loc[:,'jw_trade1'] = data[['jw_trade1a', 'jw_trade1b']].max(axis= 1)
data.loc[:,'jw_trade2'] = data[['jw_trade2a', 'jw_trade2b']].max(axis= 1)

In [ ]:
data['delta_12'] = abs(data['jw_trade1']-data['jw_trade2'])


In [ ]:
data.loc[:,['jw_trade2','jw_trade2a','jw_trade2b','BUS_NAME2clean_b','BUS_NAME2clean']].head(5)


In [ ]:
data_good = data.copy()


In [ ]:
data_good['rank1'].value_counts().head(50)


In [ ]:
data_good.loc[(data_good['BUS_NAME_1'])]


In [ ]:
data_good.loc[(data_good['BUS_NAME_1'].notnull())&\
              (data_good['DATA_SOURCE']== 'MULTI_BR'),'contains_1'] =  data_good.loc[(data_good['BUS_NAME_1'].notnull())&\
                                                                       (data_good['BUS_NAME_clean'].notnull()) &\
              (data_good['DATA_SOURCE']== 'MULTI_BR')].apply(lambda row: row['PRIMARY_BUSINESS_NAME_GAA_B'] in row['BUS_NAME_1cleanb'],axis = 1)


data_good.loc[(data_good['BUS_NAME_2'].notnull())&\
              (data_good['DATA_SOURCE']== 'MULTI_BR'),'contains_2'] =  data_good.loc[(data_good['BUS_NAME_1'].notnull())&\
                                                                       (data_good['BUS_NAME_clean'].notnull()) &\
              (data_good['DATA_SOURCE']== 'MULTI_BR')].apply(lambda row: row['PRIMARY_BUSINESS_NAME_GAA_B'] in row['BUS_NAME_2cleanb'],axis = 1)

In [ ]:
data_good.drop(columns = ['BUS_NAME_clean_b','BUS_NAME1clean_b','BUS_NAME_2clean_b'],inplace=True)


In [ ]:
data_good.columns


In [ ]:
data_good.drop(columns = ['jw_trade1a',
                         'jw_trade2a', 'jw_trade1b', 'jw_trade2b'],inplace= True)

In [ ]:
data_good.loc[:,'commonwords_1']= data_good[(data_good['BUS_NAME_1clean'].notnull())&(data_good['BUS_NAME_clean'].notnull())].apply(lambda row: calc_set_intersection(row['BUS_NAME_1clean'],row['BUS_NAME_clean']),axis=1)
data_good.loc[:,'commonwords_2']= data_good[(data_good['BUS_NAME_2clean'].notnull())&(data_good['BUS_NAME_clean'].notnull())].apply(lambda row: calc_set_intersection(row['BUS_NAME_2clean'],row['BUS_NAME_clean']),axis=1)

In [ ]:
data_good.loc[(data_good['BUS_NAME_1'].notnull())&\
              (data_good['DATA_SOURCE']!= 'MULTI_BR'),'contains_1'] =  data_good.loc[(data_good['BUS_NAME_1'].notnull())&\
                                                                       (data_good['BUS_NAME_clean'].notnull()) &\
              (data_good['DATA_SOURCE']!= 'MULTI_BR')].apply(lambda row: row['BUS_NAME_clean'] in row['BUS_NAME_1clean'],axis = 1)


data_good.loc[(data_good['BUS_NAME_2'].notnull())&\
              (data_good['DATA_SOURCE']!= 'MULTI_BR'),'contains_2'] =  data_good.loc[(data_good['BUS_NAME_2'].notnull())&\
                                                                       (data_good['BUS_NAME_clean'].notnull()) &\
              (data_good['DATA_SOURCE']!= 'MULTI_BR')].apply(lambda row: row['BUS_NAME_clean'] in row['BUS_NAME_2clean'],axis = 1)


In [ ]:
data_good.loc[data_good['contains_1'].notnull(),['DATA_SOURCE','BUS_NAME','BUS_NAME_2clean','jw_trade1','jw_trade2','contains_1','contains_2']].head(50)

In [ ]:
data_good['contains_1'].fillna(0,inplace=True)
data_good['contains_2'].fillna(0,inplace=True)


In [ ]:
def calc_set_intersection(text_a,text_b):
    try:
        text_a = str(text_a)
        a = set(text_a.split())
        b = set(text_b.split())
        return len(a.intersection(b))* 1.0
    
    except:
        return 0
    
def calc_set_single(text_a,text_b):
    
    try:
        text_a = " ".join([w for w in text_a.split() if len(w) > 1])
        text_b = " ".join([w for w in text_b.split() if len(w) > 1])
        
        a = set(text_a.split())
        b = set(text_b.split())
        
        return len(a.intersection(b))* 1.0
    
    except:
        
        return 0

def calc_set_diff(text_a,text_b,inter):
    
    try:
        
        text_a = str(text_a)
        text_b = str(text_b)
        
        a = set(text_a.split())
        b = set(text_b.split())
        
        if (inter >= 1) and (len(a) >= 2) and (len(b)>= 2):
            
            return ' '.join([w for w in list(a-b)])
        
        else:
            
            return 'no_diff'
    
    except:
        
        return 0
    

def check_middle(diff_I,diff_R,FILG_OWN_FORMAT2,name_profile_code):
    if (FILG_OWN_FORMAT2 != 'BN') and (name_profile_code == 0):
        if (diff_I != 0) and (diff_R != 0):
            if (len(diff_I)>1) & (len(diff_R) > 1):
                
                return diff_I == diff_R
            
            elif (len(diff_I)>1) | (len(diff_R) > 1):
                
                return diff_I[0] == diff_R[0]
            
            else:
                return 2
        
        else:
            
            return 99
    
    else:
        
        return 98
    

In [ ]:
data_good['BUS_NAME_clean'] = data_good['BUS_NAME_clean'].replace(' L L C','LLC')

In [ ]:
data_good.columns

In [ ]:
print(datetime.datetime.now())
data_good.loc[:,'inter0_2'] = data_good.apply(lambda row: calc_set_intersection(row['PRIMARY_BUSINESS_NAME_GAA_B_orig'],row['BUS_NAME_2clean']),axis = 1)
print(datetime.datetime.now())


print(datetime.datetime.now())
data_good.loc[:,'nosingle0_2'] = data_good.apply(lambda row: calc_set_nosingle(row['PRIMARY_BUSINESS_NAME_GAA_B_orig'],row['BUS_NAME_2clean']),axis = 1)
print(datetime.datetime.now())

print(datetime.datetime.now())
data_good.loc[:,'diff_I02'] = data_good.apply(lambda row: calc_set_diff(row['PRIMARY_BUSINESS_NAME_GAA_B_orig'],row['BUS_NAME_2clean'],row['inter0_2']),axis =1)
print(datetime.datetime.now())

print(datetime.datetime.now())
data_good.loc[:,'diff_R02'] = data_good.apply(lambda row: calc_set_diff(row['BUS_NAME_2clean'],row['PRIMARY_BUSINESS_NAME_GAA_B_orig'],row['inter0_2']),axis = 1)
print(datetime.datetime.now())


print(datetime.datetime.now())
data_good.loc[:,'inter02_02'] = data_good.apply(lambda row: calc_set_intersection(row['PRIMARY_BUSINESS_NAME_GAA_B_orig'],row['BUS_NAME_2clean']),axis = 1)
print(datetime.datetime.now())


print(datetime.datetime.now())
data_good.loc[:,'diff_R02_02'] = data_good.apply(lambda row: calc_set_diff(row['BUS_NAME_2clean'],row['PRIMARY_BUSINESS_NAME_GAA_B_orig'],row['inter02_2']),axis = 1)
print(datetime.datetime.now())

In [ ]:
print(datetime.datetime.now())
data_good.loc[:,'inter0_1'] = data_good.apply(lambda row: calc_set_intersection(row['PRIMARY_BUSINESS_NAME_GAA_B_orig'],row['BUS_NAME_1clean']),axis = 1)
print(datetime.datetime.now())


print(datetime.datetime.now())
data_good.loc[:,'nosingle0_1'] = data_good.apply(lambda row: calc_set_nosingle(row['PRIMARY_BUSINESS_NAME_GAA_B_orig'],row['BUS_NAME_1clean']),axis = 1)
print(datetime.datetime.now())

print(datetime.datetime.now())
data_good.loc[:,'diff_I01'] = data_good.apply(lambda row: calc_set_diff(row['PRIMARY_BUSINESS_NAME_GAA_B_orig'],row['BUS_NAME_1clean'],row['inter0_1']),axis =1)
print(datetime.datetime.now())

print(datetime.datetime.now())
data_good.loc[:,'diff_R01'] = data_good.apply(lambda row: calc_set_diff(row['BUS_NAME_1clean'],row['PRIMARY_BUSINESS_NAME_GAA_B_orig'],row['inter0_1']),axis = 1)
print(datetime.datetime.now())


print(datetime.datetime.now())
data_good.loc[:,'inter01_01'] = data_good.apply(lambda row: calc_set_intersection(row['PRIMARY_BUSINESS_NAME_GAA_B_orig'],row['BUS_NAME_1clean']),axis = 1)
print(datetime.datetime.now())


print(datetime.datetime.now())
data_good.loc[:,'diff_R01_01'] = data_good.apply(lambda row: calc_set_diff(row['BUS_NAME_1clean'],row['PRIMARY_BUSINESS_NAME_GAA_B_orig'],row['inter01_1']),axis = 1)
print(datetime.datetime.now())

In [ ]:
print(datetime.datetime.now())
data_good.loc[:,'inter1'] = data_good.apply(lambda row: calc_set_intersection(row['BUS_NAME_clean'],row['BUS_NAME_1clean']),axis = 1)
print(datetime.datetime.now())


print(datetime.datetime.now())
data_good.loc[:,'nosingle1'] = data_good.apply(lambda row: calc_set_nosingle(row['BUS_NAME_clean'],row['BUS_NAME_1clean']),axis = 1)
print(datetime.datetime.now())

print(datetime.datetime.now())
data_good.loc[:,'diff_I1'] = data_good.apply(lambda row: calc_set_diff(row['BUS_NAME_clean'],row['BUS_NAME_1clean'],row['inter1']),axis =1)
print(datetime.datetime.now())

print(datetime.datetime.now())
data_good.loc[:,'diff_R1'] = data_good.apply(lambda row: calc_set_diff(row['BUS_NAME_1clean'],row['BUS_NAME_clean'],row['inter1']),axis = 1)
print(datetime.datetime.now())


print(datetime.datetime.now())
data_good.loc[:,'inter2_1'] = data_good.apply(lambda row: calc_set_intersection(row['BUS_NAME_clean'],row['BUS_NAME_1clean']),axis = 1)
print(datetime.datetime.now())


print(datetime.datetime.now())
data_good.loc[:,'diff_R2_1'] = data_good.apply(lambda row: calc_set_diff(row['BUS_NAME_1clean'],row['BUS_NAME_clean'],row['inter2_1']),axis = 1)
print(datetime.datetime.now())

In [ ]:
print(datetime.datetime.now())
data_good.loc[:,'inter2'] = data_good.apply(lambda row: calc_set_intersection(row['BUS_NAME_clean'],row['BUS_NAME_2clean']),axis = 1)
print(datetime.datetime.now())


print(datetime.datetime.now())
data_good.loc[:,'nosingle'] = data_good.apply(lambda row: calc_set_nosingle(row['BUS_NAME_clean'],row['BUS_NAME_2clean']),axis = 1)
print(datetime.datetime.now())

print(datetime.datetime.now())
data_good.loc[:,'diff_I2'] = data_good.apply(lambda row: calc_set_diff(row['BUS_NAME_clean'],row['BUS_NAME_2clean'],row['inter0_1']),axis =1)
print(datetime.datetime.now())

print(datetime.datetime.now())
data_good.loc[:,'diff_R2'] = data_good.apply(lambda row: calc_set_diff(row['BUS_NAME_2clean'],row['BUS_NAME_clean'],row['inter2']),axis = 1)
print(datetime.datetime.now())


print(datetime.datetime.now())
data_good.loc[:,'inter2_2'] = data_good.apply(lambda row: calc_set_intersection(row['BUS_NAME_clean'],row['BUS_NAME_2clean']),axis = 1)
print(datetime.datetime.now())


print(datetime.datetime.now())
data_good.loc[:,'diff_R2_2'] = data_good.apply(lambda row: calc_set_diff(row['BUS_NAME_2clean'],row['BUS_NAME_clean'],row['inter2_2']),axis = 1)
print(datetime.datetime.now())

In [ ]:
data_good = data_good.sort_values(by = ['PR_DUNS','DATA_SOURCE'])


In [ ]:
print(data_good.shape)

In [ ]:
print(len(set(data_good['PR_DUNS'])))

In [ ]:
batch_match_gby = pd.DataFrame(data_good.groupby(['PR_DUNS','DATA_SOURCE']))['BUS_NAME'].count()
batch_match_gby.columns = ['count']
batch_match_gby.reset_index(drop = False,inplace=True)

In [ ]:
batch_match_gby_filter = batch_match_gby.loc[(batch_match_gby['DATA_SOURCE']=='BATCHMATCH')&\
                                             (batch_match_gby['count']>=2),'PR_DUNS']

In [ ]:
len(batch_match_gby_filter)

In [ ]:
def row_wise_1(row):
    
    if ((row['contains_1']==True) and (row['contains_2']==True)):
        return 0
    
    elif ((row['contains_1']==True) and (row['contains_2']==False)):
        return 1
    
    elif row['diff_I1'] in ['I', 'II', 'III', 'IV','V','VI','VII','VIII','IX',\
                           'X','XI','XII','XIII','XIV','XV','XVI','XVII','XVIII',\
                           'XIX','XX','ONE','TWO','THREE','FOUR','FIVE',\
                           '1','2','3','4','5','6','7','8','9']
        
        return 0
    
    elif (row['diff_I1'] == "" ) & (row['diff_I2'] == "") &\
          (row['diff_R1'] in ['I', 'II', 'III', 'IV','V','VI','VII','VIII','IX',\
                           'X','XI','XII','XIII','XIV','XV','XVI','XVII','XVIII',\
                           'XIX','XX','ONE','TWO','THREE','FOUR','FIVE',\
                           '1','2','3','4','5','6','7','8','9']) & \
          (row['diff_R2'] in ['I', 'II', 'III', 'IV','V','VI','VII','VIII','IX',\
                           'X','XI','XII','XIII','XIV','XV','XVI','XVII','XVIII',\
                           'XIX','XX','ONE','TWO','THREE','FOUR','FIVE',\
                           '1','2','3','4','5','6','7','8','9']):
    
         return 0   
    
    elif ((row['contains_1'] == False) and (row['contains_2']== False)) and \
          ((row['jw_trade1'])> (row['jw_trade2'])) and (row['jw_trade1'] >0.95):
         return 1
    

In [ ]:
def row_wise_2(row):
    
    if ((row['contains_1']==True) and (row['contains_2']==True)):
        return 0
    
    elif ((row['contains_1']==False) and (row['contains_2']==False)):
        return 1
    
    elif row['diff_I2'] in ['I', 'II', 'III', 'IV','V','VI','VII','VIII','IX',\
                           'X','XI','XII','XIII','XIV','XV','XVI','XVII','XVIII',\
                           'XIX','XX','ONE','TWO','THREE','FOUR','FIVE',\
                           '1','2','3','4','5','6','7','8','9']
        
        return 0
    
    elif (row['diff_I1'] == "" ) & (row['diff_I2'] == "") &\
          (row['diff_R1'] in ['I', 'II', 'III', 'IV','V','VI','VII','VIII','IX',\
                           'X','XI','XII','XIII','XIV','XV','XVI','XVII','XVIII',\
                           'XIX','XX','ONE','TWO','THREE','FOUR','FIVE',\
                           '1','2','3','4','5','6','7','8','9']) & \
          (row['diff_R2'] in ['I', 'II', 'III', 'IV','V','VI','VII','VIII','IX',\
                           'X','XI','XII','XIII','XIV','XV','XVI','XVII','XVIII',\
                           'XIX','XX','ONE','TWO','THREE','FOUR','FIVE',\
                           '1','2','3','4','5','6','7','8','9']):
    
         return 0 
    
    elif ((row['contains_1'] == False) and (row['contains_2']== False)) and \
          ((row['jw_trade1'])< (row['jw_trade2'])) and (row['jw_trade2'] >0.95):
         return 1
    
    
    
    

In [ ]:
data_good['final_1'] = data_good.apply(lambda row: row_wise_1(row),axis = 1)
data_good['final_2'] = data_good.apply(lambda row: row_wise_2(row),axis = 1)


In [ ]:
data_good['final_1'].fillna(0,inplace=True)
data_good['final_2'].fillna(0,inplace=True)


In [ ]:
alldata_gby = data_good[(data_good['DATA_SOURCE']!= 'MULTI_BR')].groupby(['PR_DUNS'])['final_1','final_2'].sum()
tradedata_gby = data_good[(data_good['DATA_SOURCE']== 'TRADE')].groupby(['PR_DUNS'])['final_1','final_2'].sum()
BATCHMATCH_gby = data_good[(data_good['DATA_SOURCE']!= 'ZBATCHMATCH')].groupby(['PR_DUNS'])['final_1','final_2'].sum()

alldata_gby.reset_index(drop = False,inplace=True)
tradedata_gby.reset_index(drop = False,inplace=True)
BATCHMATCH_gby.reset_index(drop = False,inplace=True)

In [ ]:
print(alldata_gby.shape)
alldata_gby = pd.merge(alldata_gby,tradedata_gby, on = 'PR_DUNS', how = 'left',\
                      suffixes = ['','_trade'])

alldata_gby = pd.merge(alldata_gby,BATCHMATCH_gby, how = 'left',\
                      suffixes = ['','_bm'])
print(alldata_gby.shape)

In [ ]:
alldata_gby.fillna(0,inplace=True)


In [ ]:
alldata_gby['final_12_temp'] = alldata_gby['final_1'] - alldata_gby['final_2']
alldata_gby['diff'] = alldata_gby['final_12_temp'].min(axis = 1)
alldata_gby.drop(columns = ['final_12_temp'],inplace=True)

In [ ]:
alldata_gby['diff'].value_counts().head(50)

In [ ]:
multi_data_good = pd.DataFrame(data_good[data_good['DATA_SOURCE']=='MULTI_BR']).groupby()
multi_data_good.reset_index(drop = False,inplace=True)
multi_data_good.columns = ['PR_DUNS','count']
print(multi_data_good.shape)
multi_data_good_all = multi_data_good.copy()
multi_data_good = multi_data_good[multi_data_good['count']==2]
print(multi_data_good.shape)

In [ ]:
multi_data_good_all['count'].value_counts()

In [ ]:
DELINQUENT_duns = set((data_good[(data_good['DATA_SOURCE']=='MULTI_BR')&(data_good['BUS_NAME'].str.contains('DELINQUENT')),'PR_DUNS'])
print(len(DELINQUENT_duns))

In [ ]:
unresolved_duns1 = set(data_good.loc[(data_good['DATA_SOURCE']=='MULTI_BR')&(data_good['final_12']==0),'PR_DUNS'])
print(len(unresolved_duns1))

In [ ]:
data_good['final_12'] = data_good['final_1']+data_good['final_2']

In [ ]:
print(data_good.shape)
data_good_1 = data_good[data_good['PR_DUNS'].isin(multi_data_good['PR_DUNS'])]
print(data_good_1.shape)

data_good_1.reset_index(drop = True,inplace=True)

print(data_good_1.shape)
data_good_1 = data_good[~data_good['PR_DUNS'].isin(DELINQUENT_duns)]
print(data_good_1.shape)
data_good_1.reset_index(drop = True,inplace=True)

print(data_good_1.shape)
data_good_1 = data_good[~data_good['PR_DUNS'].isin(unresolved_duns1)]
print(data_good_1.shape)
data_good_1.reset_index(drop = True,inplace=True)




In [ ]:
def final_decision1(row):
    
    if row['diff']>= 0:
        return 'No Name Change'
    elif (row['diff']<-5)&(row['final_1']>-1):
        return 'Change'
    else:
        return 'Extra'

In [ ]:
alldata_gby['decision'] = alldata_gby.apply(lambda row: final_decision(row),axis = 1)

In [ ]:
alldata_gby['decision'].value_counts()

In [ ]:
print(data_good_1.shape)
data_good_1 = data_good_1[data_good_1['PR_DUNS'].isin(alldata_gby.loc[alldata_gby['decision']!='Change','PR_DUNS'])]
print(data_good_1.shape)
data_good_1.reset_index(drop = True,inplace=True)

In [ ]:
data_good.to_csv('data_trade_move_corr1.csv',index=False)
data_bad.to_csv('data_trade_move_bad_corr1.csv',index=False)
data_good_1.to_csv('data_trade_move_v1_corr1.csv',index=False)



In [ ]:
list(data_good_1.columns)